# Water security project for Modern Data Analytics 

## 1. Water gathering and preprocessing

#### The data was downloaded from the Aqueduct 3.0: Updated Decision-Relevant Global Water Risk Indicators provided by the World Resources Institute. The report can be found in https://www.wri.org/research/aqueduct-30-updated-decision-relevant-global-water-risk-indicators

The metadata can be found here https://github.com/wri/aqueduct30_data_download/blob/master/metadata.md

In [70]:
import pandas as pd

#df_w = pd.read_csv("data/y2019m07d11_aqueduct30_annual_v01.csv")

In [71]:
# Keep the relevant columns regarding water stress, water depletion and risks.
#df_w = df_w[df_w.columns[:60]]

## 2. United Nations UN water data

Using the United Nations Water webpage, we can build a dataframe that brings global indicators regarding social, economical and enviromental indicators. Even if the webpage is itself an interactive map, we can make use of the API that the browser uses to get the data. By taking a look at how the browser would make use of the API, we would do it the same way by changing some parameters. Later, we would analyze the data using the tools reviewed in the lectures

#### 2.1 Data fetch

When we open the webpage https://sdg6data.org/data-lab, we can filter by which indicators we want to see in the map and in which region. With the use of the browser tools, it is possible to get the cUrl that the browser does. After some attempts, it is possible to see the url that the API uses and some of the parameters that change in each request. This parameters will be stored in _indicators_ so we can loop the same process to build a dataframe

In [72]:
import requests

# 6.1.1 Proportion of population using safely managed drinking water services
url = "https://www.sdg6data.org/handle-ajax-callback"

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:103.0) Gecko/20100101 Firefox/103.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Requested-With": "XMLHttpRequest",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin"
}

indicators = [
    ("79_80", "0.00", "3,850.49", "stress"), # Water stress
    ("82_83", "0.00", "100.00", "mngmt"), # Water resource management
    ("84_85", "0.00", "100.00", "basin"), # Transboundary basin area
    ("135_137_138", "0.00", "8,647.00", "renewable"), # Renewable water
    ("1_2_3_4", "0.00", "100.00", "safe"), # Proportion of population using safely managed drinking water services
    ("483_508", "731.06", "112,226.90", "gdp"), # GDP per capita
    ("483_503", "0.00", "100.0", "elec"), # Proportion of population with access to electricity
    ("484_492", "0.00", "100.00", "poverty"), # Poverty
    ("485_499", "0.00", "100.00", "agr") # Agricultural land 
]

The following function will receive 4 arguments to build a __df__. It will create the _body_ needed for the request and it will return the __df__ with the information by country

In [76]:
def generate_df(indicator, min_bound, max_bound, col_name):
    r"""
    So after taking a look at the API that the UN Water uses to get the data, it asks for the values regarding the locations, the number of the indicator and the ranges.
    Given that the URL and the headers are the same, there are a few parameters that change between each request
    
    Keyword arguments:
    indicator -- is like the id of the indicator desired to be fetch
    min_bound -- is the minimum value possible to get. In the website it is possible to filter by this value
    max_bound -- is the maximum value possible to get. In the website it is possible to filter by this value
    col_name -- is the name of the column in the pd.DataFrame
    """

    body = f"allRegion%5B%5D=World--World&allRegion%5B%5D=Geographical+region--Latin+America+and+the+Caribbean&allRegion%5B%5D=Geographical+region--Asia&allRegion%5B%5D=Geographical+region--Europe&allRegion%5B%5D=Geographical+region--Northern+Africa&allRegion%5B%5D=Geographical+region--Northern+America&allRegion%5B%5D=Geographical+region--Oceania&allRegion%5B%5D=Geographical+region--Sub-Saharan+Africa&indicators%5B%5D={indicator}&ranges%5B0%5D%5B%5D={min_bound}&ranges%5B0%5D%5B%5D={max_bound}"
    req = requests.post(url, headers=headers, data=body)
    data = req.json()["commonCountriesDataTable"]
    df = pd.DataFrame()
    df["country"] = data.keys()
    df[col_name] = data.values()
    return df

_indicators_ will be looped using the function created and it will be joined to a first __df__ created using the _water stress_ variable given that it is an important variable for the analysis. The result will be a dataframe with 9 variable of 179 countries

In [77]:
ind, min, max, col_name = indicators[0]
df = generate_df(ind, min, max, col_name)

for ind, min, max, col_name in indicators[1:]:
    df_1 = generate_df(ind, min, max, col_name)
    df = pd.merge(df, df_1, how="left", on=["country", "country"])